In [319]:
#Steve Branch

import pandas as pd
import numpy as np

In [320]:
#Load the data file
mental_health_data_filename = 'mental_health_data_true.csv'
mental_health_df= pd.read_csv(mental_health_data_filename)

question_cross_ref_filename = 'mental_health_header_cross_ref.csv'
question_cross_ref_df = pd.read_csv(question_cross_ref_filename)

In [321]:
# Remove Self Employed Records from the Data

# Two ways to do this: Method 1:
# Set the Dataframe equal to record where data['A'] == 0
# data = data[data['A'] == 0]
# data.shape

# Method 2:
# Actually drop the records from the data frame using frop
mental_health_df.drop(mental_health_df[mental_health_df['A'] == 1].index, inplace = True)
mental_health_df.shape

(1146, 63)

In [322]:
#Drop columns

In [323]:
#Combine WW and XX

In [324]:
#Column XX and WW missing values

In [325]:
#Column XX. WW, ZZ codify non-standard values as other

In [326]:
#Replace General Missing Values

In [327]:
#Handle Gender Somehow
# This var makes my life a little easier
# TODO: gender_col_id may change after integration, double check this
gender_col_id = "EEE"

# for this method to work beter, strip leading and ending whitespace
mental_health_df[gender_col_id] = mental_health_df[gender_col_id].str.strip()


# regex explanation:
# (?i) - case insensitive, looks for female or woman or f
female_regex_pattern = r'(?i)(female|woman|f)'
# regex explanation:
# (?i) - case insensitive, looks for male or man or m
male_regex_pattern = r'(?i)(male|man|m)'

# regex explanation:
# any value, not F or M gets O
other_regex_pattern = r'^(?!M)^(?!F)'

# replace female
mental_health_df[gender_col_id].replace(to_replace=female_regex_pattern, value = 'F',
                                        inplace = True, regex= True)
# replace male
mental_health_df[gender_col_id].replace(to_replace=male_regex_pattern, value = 'M',
                                        inplace = True, regex= True)
# # replace other
mental_health_df[gender_col_id].replace(to_replace=other_regex_pattern, value = 'O',
                                        inplace = True, regex= True)

print(mental_health_df[gender_col_id].unique())
#

['M' 'F' 'OI identiFy as F.' 'OBigender' 'Onon-binary'
 'F assigned at birth' 'FM' 'OCis F' 'OTransitioned, M2F'
 'OGenderFluid (born F)' 'OOther/TransFeMinine' 'F or Multi-Gender FeMMe'
 'F/F' 'OCis M' 'M.' 'OAndrogynous' 'M 9:1 F, roughly' nan 'M (cis)'
 'OOther' 'Onb Masculine' 'OCisgender F' 'OSex is M'
 'Onone oF your business' 'Ogenderqueer' 'Ocis M' 'OHuM' 'OGenderFluid'
 'OEnby' 'Ogenderqueer F' 'OQueer' 'OAgender' 'ODude' 'Mail'
 'M/genderqueer' 'FeM' 'ONonbinary'
 'F (props For Making this a FreeForM Field, though)' 'M (trans, FtM)'
 'OCis-F' 'OGenderqueer' 'Ocisdude' 'OGenderFlux deMi-girl'
 'F-bodied; no Feelings about gender' 'OTransgender F']


In [328]:
#Age Outliers

In [329]:
#Encode yes,no,ect values 

In [330]:
# TODO: We probably should write the data to a clean csv file
# DO NOT OVER WRITE EXISTING CSV, it should be written to a new file
